In [1]:
# ===================================================================
#  Library
# ===================================================================
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error

import warnings
warnings.simplefilter("ignore")

import optuna
import numpy as np

In [7]:
# ===================================================================
#  CFG
# ===================================================================
class CFG:
    seed = 42
    n_seeds = 3
    n_trials = 2000
    save_dir = "G:/マイドライブ/signate_StudentCup2023/exp/"
    data_dir = "G:/マイドライブ/signate_StudentCup2023/data/"
    filename = "exp049"

In [3]:
# ===================================================================
#  Utils
# ===================================================================
def get_score(y_true, y_pred):
    """get MAPE score"""
    score = mean_absolute_percentage_error(y_true, y_pred)
    return score * 100

In [8]:
# ===================================================================
#  DataLoading
# ===================================================================
df_1 = pd.read_csv(CFG.save_dir+"oof_df_exp039.csv", names=['id', 'pred']).rename(columns={"pred":"pred_1"})
df_2 = pd.read_csv(CFG.save_dir+"oof_df_exp040.csv", names=['id', 'pred']).rename(columns={"pred":"pred_2"})
df_3 = pd.read_csv(CFG.save_dir+"oof_df_exp041.csv", names=['id', 'pred']).rename(columns={"pred":"pred_3"})
df_4 = pd.read_csv(CFG.save_dir+"oof_df_exp042.csv", names=['id', 'pred']).rename(columns={"pred":"pred_4"})
df_5 = pd.read_csv(CFG.save_dir+"oof_df_exp043.csv", names=['id', 'pred']).rename(columns={"pred":"pred_5"})
df_6 = pd.read_csv(CFG.save_dir+"oof_df_exp044.csv", names=['id', 'pred']).rename(columns={"pred":"pred_6"})
df_7 = pd.read_csv(CFG.save_dir+"oof_df_exp045.csv", names=['id', 'pred']).rename(columns={"pred":"pred_7"})
df_8 = pd.read_csv(CFG.save_dir+"oof_df_exp046.csv", names=['id', 'pred']).rename(columns={"pred":"pred_8"})
df_9 = pd.read_csv(CFG.save_dir+"oof_df_exp047.csv", names=['id', 'pred']).rename(columns={"pred":"pred_9"})
df_10 = pd.read_csv(CFG.save_dir+"oof_df_exp048.csv", names=['id', 'pred']).rename(columns={"pred":"pred_10"})

train = pd.read_csv(CFG.data_dir+"train.csv")

df = pd.merge(train, df_1[["id", "pred_1"]], on="id", how="left")
df = pd.merge(df, df_2[["id","pred_2"]], on="id", how="left")
df = pd.merge(df, df_3[["id", "pred_3"]], on="id", how="left")
df = pd.merge(df, df_4[["id", "pred_4"]], on="id", how="left")
df = pd.merge(df, df_5[["id", "pred_5"]], on="id", how="left")
df = pd.merge(df, df_6[["id", "pred_6"]], on="id", how="left")
df = pd.merge(df, df_7[["id", "pred_7"]], on="id", how="left")
df = pd.merge(df, df_8[["id", "pred_8"]], on="id", how="left")
df = pd.merge(df, df_9[["id", "pred_9"]], on="id", how="left")
df = pd.merge(df, df_10[["id", "pred_10"]], on="id", how="left")

df.to_csv(CFG.save_dir+f"{CFG.filename}_preds.csv", index=False)

In [5]:
# ===================================================================
#  optuna
# ===================================================================
def objective(trial):
    a = trial.suggest_float("a", 1e-8, 1, log=True)
    b = trial.suggest_float("b", 1e-8, 1, log=True)
    c = trial.suggest_float("c", 1e-8, 1, log=True)
    d = trial.suggest_float("d", 1e-8, 1, log=True)
    e = trial.suggest_float("e", 1e-8, 1, log=True)
    
    f = trial.suggest_float("f", 1e-8, 1, log=True)
    g = trial.suggest_float("g", 1e-8, 1, log=True)
    h = trial.suggest_float("h", 1e-8, 1, log=True)
    i = trial.suggest_float("i", 1e-8, 1, log=True)
    j = trial.suggest_float("j", 1e-8, 1, log=True)
        
    df[f"pred"] = df[f"pred_1"] * a +\
                  df[f"pred_2"] * b +\
                  df[f"pred_3"] * c +\
                  df[f"pred_4"] * d +\
                  df[f"pred_5"] * e +\
                  df[f"pred_6"] * f +\
                  df[f"pred_7"] * g +\
                  df[f"pred_8"] * h +\
                  df[f"pred_9"] * i +\
                  df[f"pred_10"] * j
                      
    score = get_score(y_true=df["price"], y_pred = df[f"pred"])
    return score
    
optuna.logging.set_verbosity(optuna.logging.WARNING)

# シードのリストを定義
seeds = [seed for seed in range(CFG.seed, CFG.seed+CFG.n_seeds)]

best_values = []
best_params_list = []

for seed in seeds:    
    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(seed=seed)
    )
    study.optimize(objective, 
                   n_trials=CFG.n_trials, 
                   n_jobs = -1,
                   show_progress_bar=True)
    
    best_value = study.best_value
    best_params = study.best_params
    
    best_values.append(best_value)
    best_params_list.append(best_params)
    
    print(f"Seed: {seed}, Best Value: {best_value}, Best Params: {best_params}")
    
    
# 最も小さい best_value を持つ Study を探索
best_index = np.argmin(best_values)
best_params_final = best_params_list[best_index]
best_value_final = best_values[best_index]

print("Final Best Value:", best_value_final)
print("Final Best Params:", best_params_final)

  0%|          | 0/2000 [00:00<?, ?it/s]

Seed: 42, Best Value: 43.69660836605411, Best Params: {'a': 0.08401399371625468, 'b': 3.859086237561503e-07, 'c': 1.6319696123104263e-07, 'd': 4.9547716121534974e-05, 'e': 0.2728216342479428, 'f': 0.12452781061938452, 'g': 0.019077681313046938, 'h': 4.709963461174562e-07, 'i': 0.48796610167821575, 'j': 0.006755072484201945}


  0%|          | 0/2000 [00:00<?, ?it/s]

Seed: 43, Best Value: 43.698466398799766, Best Params: {'a': 0.14412962262192655, 'b': 1.6881492268792355e-07, 'c': 1.4271902131530857e-05, 'd': 0.3196650848987985, 'e': 0.04795999366912311, 'f': 7.227674469787484e-07, 'g': 5.947123785395577e-05, 'h': 0.0009460853888719193, 'i': 0.4820174113831951, 'j': 1.1234391824783612e-05}


  0%|          | 0/2000 [00:00<?, ?it/s]

Seed: 44, Best Value: 43.71171899595376, Best Params: {'a': 0.2420841457443737, 'b': 0.006129852000482812, 'c': 2.538324417113941e-06, 'd': 7.995282925707243e-06, 'e': 0.1330573094480169, 'f': 5.890793951345941e-07, 'g': 5.385571716577776e-08, 'h': 0.03839078613358119, 'i': 0.5794520458002477, 'j': 9.970556101170305e-08}
Final Best Value: 43.69660836605411
Final Best Params: {'a': 0.08401399371625468, 'b': 3.859086237561503e-07, 'c': 1.6319696123104263e-07, 'd': 4.9547716121534974e-05, 'e': 0.2728216342479428, 'f': 0.12452781061938452, 'g': 0.019077681313046938, 'h': 4.709963461174562e-07, 'i': 0.48796610167821575, 'j': 0.006755072484201945}


In [9]:
# ===================================================================
#  test
# ===================================================================
test_1 = pd.read_csv(CFG.save_dir+"exp039.csv", names=['id', 'pred']).rename(columns={"pred":"pred_1"})
test_2 = pd.read_csv(CFG.save_dir+"exp040.csv", names=['id', 'pred']).rename(columns={"pred":"pred_2"})
test_3 = pd.read_csv(CFG.save_dir+"exp041.csv", names=['id', 'pred']).rename(columns={"pred":"pred_3"})
test_4 = pd.read_csv(CFG.save_dir+"exp042.csv", names=['id', 'pred']).rename(columns={"pred":"pred_4"})
test_5 = pd.read_csv(CFG.save_dir+"exp043.csv", names=['id', 'pred']).rename(columns={"pred":"pred_5"})
test_6 = pd.read_csv(CFG.save_dir+"exp044.csv", names=['id', 'pred']).rename(columns={"pred":"pred_6"})
test_7 = pd.read_csv(CFG.save_dir+"exp045.csv", names=['id', 'pred']).rename(columns={"pred":"pred_7"})
test_8 = pd.read_csv(CFG.save_dir+"exp046.csv", names=['id', 'pred']).rename(columns={"pred":"pred_8"})
test_9 = pd.read_csv(CFG.save_dir+"exp047.csv", names=['id', 'pred']).rename(columns={"pred":"pred_9"})
test_10 = pd.read_csv(CFG.save_dir+"exp048.csv", names=['id', 'pred']).rename(columns={"pred":"pred_10"})

test = pd.merge(test_1, test_2, on="id", how="left")
test = pd.merge(test, test_3, on="id", how="left")
test = pd.merge(test, test_4, on="id", how="left")
test = pd.merge(test, test_5, on="id", how="left")
test = pd.merge(test, test_6, on="id", how="left")
test = pd.merge(test, test_7, on="id", how="left")
test = pd.merge(test, test_8, on="id", how="left")
test = pd.merge(test, test_9, on="id", how="left")
test = pd.merge(test, test_10, on="id", how="left")

test["pred"] =   test["pred_1"] * best_params_final["a"] +\
                 test["pred_2"] * best_params_final["b"] +\
                 test["pred_3"] * best_params_final["c"] +\
                 test["pred_4"] * best_params_final["d"] +\
                 test["pred_5"] * best_params_final["e"] +\
                 test["pred_6"] * best_params_final["f"] +\
                 test["pred_7"] * best_params_final["g"] +\
                 test["pred_8"] * best_params_final["h"] +\
                 test["pred_9"] * best_params_final["i"] +\
                 test["pred_10"] * best_params_final["j"]
                     
test.to_csv(CFG.save_dir+f"{CFG.filename}_test_preds.csv", index=False)                     

test[["id", "pred"]].to_csv(CFG.save_dir+f"{CFG.filename}.csv", index=False, header=None)
test[["id", "pred"]].head(2)

,id,pred
0,27532,9217.092961
1,27533,5535.468202
